<a href="https://colab.research.google.com/github/ethamCS/Bert-Medical-Apps/blob/main/initial_mhealth_app_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🤗 Oy vey




In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.9 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00


In [68]:
from datasets import load_dataset,load_from_disk
from transformers import TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer
from collections import Counter
import numpy as np
import evaluate
import numpy as np
from sklearn.metrics import classification_report

In [69]:
# ! rm -r ~/.cache/huggingface/datasets

In [70]:
test = load_dataset("csv", data_files="/content/processed_test.csv")
train = load_dataset("csv", data_files="/content/processed_train.csv")
test = test['train']
train =  train['train']

train = train.shuffle(seed=42)
test = test.shuffle(seed=42)

print(train.shape)
print(test.shape)

(500, 4)
(100, 4)


In [71]:
# import pandas as pd
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer
# import re
# import string

# nltk.download("stopwords")
# nltk.download("punkt")

# dataset = pd.read_csv('train_400.csv')

# def remove_html_tags(text):
#     clean = re.compile("<.*?>")
#     return re.sub(clean, "", text)

# def remove_urls(text):
#     url_pattern = re.compile(r"https?://\S+|www\.\S+")
#     return url_pattern.sub("", text)

# def remove_punctuation(text):
#     translator = str.maketrans("", "", string.punctuation)
#     return text.translate(translator)


# def preprocess_text(input_text):
#     text_no_html = remove_html_tags(input_text)

#     text_no_urls = remove_urls(text_no_html)

#     text_no_punctuation = remove_punctuation(text_no_urls)

#     words = word_tokenize(text_no_punctuation)

#     stop_words = set(stopwords.words("english"))
#     filtered_words = [word for word in words if word.lower() not in stop_words]

#     stemmer = PorterStemmer()
#     stemmed_words = [stemmer.stem(word) for word in filtered_words]

#     processed_text = " ".join(stemmed_words)

#     return processed_text

# dataset['description'] = dataset['description'].apply(preprocess_text)


# print(dataset.head())
# dataset.to_csv('/content/processed_train.csv')

In [72]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["description"], padding="max_length", truncation=True, max_length=512)

train_data = train.map(tokenize_function, batched=True,remove_columns= ['description','Unnamed: 0','Unnamed: 0.1'])

test_data = test.map(tokenize_function, batched=True,remove_columns= [ 'description','Unnamed: 0','Unnamed: 0.1'])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [73]:
print(train_data)
print(test_data)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 500
})
Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})


In [74]:
id2label = {0: "NON-MHEALTH", 1: "MHEALTH"}
label2id = {"NON-MHEALTH": 0, "MHEALTH": 1}

In [75]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:
accuracy = evaluate.load("accuracy")

In [77]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    print(classification_report(
        labels,
        predictions,
        zero_division = 0,
    ))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'acc':acc

    }

In [78]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [79]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
   )

In [80]:
trainer = Trainer(
  model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [81]:
# token_lengths = [len(tokens) for tokens in tokenized_datasets['train']['input_ids']]
# print("Max token length:", max(token_lengths))

In [82]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [83]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Acc
1,No log,0.136793,0.938776,0.978723,0.958333,0.960000
2,No log,0.151161,0.958333,0.978723,0.968421,0.970000
3,No log,0.142040,0.958333,0.978723,0.968421,0.970000


              precision    recall  f1-score   support

           0       0.98      0.94      0.96        53
           1       0.94      0.98      0.96        47

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100

              precision    recall  f1-score   support

           0       0.98      0.96      0.97        53
           1       0.96      0.98      0.97        47

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100

              precision    recall  f1-score   support

           0       0.98      0.96      0.97        53
           1       0.96      0.98      0.97        47

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100



TrainOutput(global_step=189, training_loss=0.13514968831703145, metrics={'train_runtime': 174.9858, 'train_samples_per_second': 8.572, 'train_steps_per_second': 1.08, 'total_flos': 394666583040000.0, 'train_loss': 0.13514968831703145, 'epoch': 3.0})

In [84]:
trainer.evaluate()

              precision    recall  f1-score   support

           0       0.98      0.96      0.97        53
           1       0.96      0.98      0.97        47

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100



{'eval_loss': 0.14203983545303345,
 'eval_precision': 0.9583333333333334,
 'eval_recall': 0.9787234042553191,
 'eval_f1': 0.968421052631579,
 'eval_acc': 0.97,
 'eval_runtime': 3.4668,
 'eval_samples_per_second': 28.845,
 'eval_steps_per_second': 3.75,
 'epoch': 3.0}

In [85]:
import torch
# snore_lab = ''' Record and track your snoring with the No.1 snore app. Sleep quieter and better!
# - Monitoring over 1 million nights of snoring per month
# - Measures how loud your snoring is and tracks it over time
# - The worldwide No.1 app for snorers on iOS and Android'''

text = ''' Shareable trip planner: Collaborate, plan and share travel information with others, all in one place.
<b>Manage your whole trip in one app:</b> - Free cancellation on most hotels.* -
All your travel reservations in one place: view upcoming flight details, hotel booking information, car rental bookings. -
Travel alerts: for flight delays, gate changes, hotel check-out times, and more. -
'''

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)
predictions = outputs.logits.squeeze().cpu()

sigmoid = torch.nn.Sigmoid()
probs = sigmoid(torch.Tensor(predictions))
predictions[np.where(probs >= 0.5)] = 1
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predictions)
print(predicted_labels)

tensor([ 1.0000, -1.6776], grad_fn=<IndexPutBackward0>)
['NON-MHEALTH']
